In [ ]:
# load all libraries

import os
import tifffile
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from natsort import natsorted
from collections import defaultdict
import cv2
from scipy.stats import linregress
from scipy.optimize import curve_fit
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

# Part 2: Get Stern-Volmer parameters for the calibrated optode

- Load calibration data from excel
- Choose the ratio data to use for calibration (RAW, 2nd degree smoothed or 3rd degree smoothed)
- Get column-by-column S-V parameters for all images
---

In [ ]:
# Set the location of the images.
Drive ="C:/"
Folder = "YOUR FOLDER"
Mearsurements = "YOUR SUB-FOLDER"

- Load calibration data from excel

In [ ]:
#  Laod excel data file and Set Oxygen value to Float
excel_file_path = f'{Drive}/{Folder}/{Mearsurements}/RAW_Calibration_data.xlsx'

data = pd.read_excel(excel_file_path)
data["Oxygen%"] = data["Oxygen%"].astype(float)
print(data.dtypes)


- Optionally create column groups to obtain average data for more columns.

In [ ]:
# Create a group key based on the Column Index divided into groups of 20
data['Group Index'] = data['Column Index'] // 30

# Group by Temp, Oxygen%, and the new Group Index to compute the mean for the ratio columns
grouped_data = data.groupby(['Temp', 'Oxygen%', 'Group Index'])[['Original Ratio', '2nd Degree Smoothed Ratio', '3rd Degree Smoothed Ratio', 'Linear Smoothed Ratio']].mean().reset_index()

# Display the first few rows of the grouped data
grouped_data

- Get R0 values for all colums or column groups.

In [ ]:
# Step 1: Create a dictionary for R0 values
# Adjust R0_dict to account for both Column Index and Temp
R0_dict = grouped_data[grouped_data['Oxygen%'] == 0].set_index(['Group Index', 'Temp'])['Linear Smoothed Ratio'].to_dict()

grouped_data['Normalized Predicted Ratio'] = grouped_data.apply(
    lambda row: R0_dict[(row['Group Index'], row['Temp'])] / row['Linear Smoothed Ratio'] if (row['Group Index'], row['Temp']) in R0_dict else np.nan,
    axis=1)

grouped_data

- Apply modified Stern-Volmer equation to model and predict fitting parameters Ksv and f for all columns (or groups) at all temperatures.
- Save parameters in a dataframe

In [ ]:
data = grouped_data  # Use your DataFrame here

# Define the modified Stern-Volmer equation
def modified_stern_volmer(x, k, f):
    return 1 / (f / (1 + k * x) + (1 - f))

initial_params = {'k': 0.165, 'f': 0.887}  # Initial parameter estimates

# Initialize a DataFrame to store parameters for each column index and temperature
all_params_df = pd.DataFrame()

# Function for prediction using the fitted polynomial models
def predict_intensity_ratio(temp, oxygen, model_k, model_f):
    k_temp = model_k.predict([[temp]])[0]
    f_temp = model_f.predict([[temp]])[0]
    return 1 / (f_temp / (1 + k_temp * oxygen) + (1 - f_temp))
# Function for visualization
def visualize_predictions_normalized(column_index, temperatures, oxygen_range, model_k, model_f, normalized_data):
    plt.figure(figsize=(12, 8))
    colors = plt.cm.viridis(np.linspace(0, 1, len(temperatures)))
    markers = ['o', 's', '^', 'd', 'x', 'p', '*']

    # Plot predicted data
    for temp, color in zip(temperatures, colors):
        predicted_ratios = [predict_intensity_ratio(temp, o2, model_k, model_f) for o2 in oxygen_range]
        plt.plot(oxygen_range, predicted_ratios, label=f'Predicted {temp}°C', color=color)

    # Plot actual data
    for temp, color, marker in zip(normalized_data['Temp'].unique(), colors, markers):
        temp_data = normalized_data[normalized_data['Temp'] == temp]
        plt.scatter(temp_data['Oxygen%'], temp_data['Normalized Predicted Ratio'], marker=marker, color=color, label=f'Actual {temp}°C')

    plt.xlabel('Air Saturation (%))')
    plt.ylabel('Normalized Predicted Ratio')
    plt.title(f'Column Index {column_index}: Model Predictions vs Actual Data')
    plt.legend()
    plt.grid(True)
    plt.savefig(f"Column_{column_index}_predictions_vs_actual.png", dpi=300)
    plt.show()

# Process each column index to compute parameters
for col_index in data['Group Index'].unique():
    column_data = data[data['Group Index'] == col_index]

    # DataFrame to hold parameters for the current column index
    parameters_df = pd.DataFrame()

    for temp in column_data['Temp'].unique():
        temp_data = column_data[column_data['Temp'] == temp]
        try:
            popt, _ = curve_fit(modified_stern_volmer, temp_data['Oxygen%'], temp_data['Normalized Predicted Ratio'], p0=[initial_params['k'], initial_params['f']], bounds=([0, 0], [np.inf, 1]), maxfev=10000)
            parameters_df = parameters_df.append({
                'Group Index': col_index,
                'Temp': temp,
                'k': popt[0],
                'f': popt[1]
            }, ignore_index=True)
        except RuntimeError as e:
            print(f"Failed to fit model for Column Index {col_index}, Temp {temp}: {str(e)}")

    # Append parameters for the current column to the master DataFrame
    all_params_df = pd.concat([all_params_df, parameters_df], ignore_index=True)

# Print or save the complete parameters DataFrame
print(all_params_df)
# all_params_df.to_csv('model_parameters.csv', index=False)  # Optionally save to CSV


- Display Stern-Volmer fit for all temperatures at two selcted column indicies 

In [ ]:
# Specify the columns you want to plot
columns_to_plot = [0, 60]  # Update this list with your desired column indices

# visualize selected column indices
for col_index in columns_to_plot:
    if col_index in data['Group Index'].unique():
        column_data = data[data['Group Index'] == col_index]
        parameters_df = all_params_df[all_params_df['Group Index'] == col_index]

        # Fit polynomial models for visualization
        model_k = make_pipeline(PolynomialFeatures(degree=2), LinearRegression())
        model_k.fit(parameters_df[['Temp']], parameters_df['k'])

        model_f = make_pipeline(PolynomialFeatures(degree=2), LinearRegression())
        model_f.fit(parameters_df[['Temp']], parameters_df['f'])

        # Visualization for selected columns
        example_temperatures = column_data['Temp'].unique()
        oxygen_range_example = np.linspace(0, 100, 10)  # Define a range of oxygen concentrations
        visualize_predictions_normalized(col_index, example_temperatures, oxygen_range_example, model_k, model_f, column_data)
    else:
        print(f"Column Index {col_index} not found in the dataset.")

- Add the R0 value to the parameters dataframe and save dataframe to excel file.

In [ ]:
# Function to fetch R0 values using both Column Index and Temp
def get_R0(row):
    key = (row['Group Index'], row['Temp'])
    return R0_dict.get(key, np.nan)  # Returns np.nan if the key is not found

# Apply the function to each row in fit_results_df to create the R0 column
all_params_df['R0'] = all_params_df.apply(get_R0, axis=1)

all_params_df.to_excel(f'{Drive}/{Folder}/{Mearsurements}/SV_Calibration_Parameters.xlsx', index=False)

all_params_df


---